# Example of geometry optimization
## Batch of two molecules

In [5]:
import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.MolecularDynamics import Geometry_Optimization_SD
from seqm.seqm_functions.read_xyz import read_xyz
from seqm.seqm_functions.save_xyz import save_xyz
torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [6]:
%%time

species = torch.as_tensor([[8,6,1,1],
                           [8,6,1,1],
                           [8,1,1,0]], # zero-padding for batching
                          dtype=torch.int64, device=device)

coordinates = torch.tensor([
                              [
                               [0.00,    0.00,    0.00],
                               [1.22,    0.00,    0.00],
                               [1.82,    0.94,    0.00],
                               [1.82,   -0.94,    0.00]
                              ],
                              [
                               [0.00,    0.00,    0.00],
                               [1.22,    0.00,    0.00],
                               [1.82,    0.94,    0.00],
                               [1.82,   -0.94,    0.00]
                              ],
                              [
                               [ 0.00,    0.00,    0.00],
                               [ 0.96,    0.00,    0.00],
                               [-0.45,   -0.57,    0.67],
                               [0.0,0.0,0.0]            # zero-padding for batching
                              ]
                            ], device=device)

const = Constants().to(device)
#may need to add scaling factor for length and energy on const, check constants.py

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'PM6_SP', # AM1, MNDO, PM3, PM6, PM6_SP. PM6_SP is PM6 without d-orbitals. Effectively, PM6 for the first two rows of periodic table
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [1,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

opt =  Geometry_Optimization_SD(seqm_parameters, alpha=0.008, force_tol=1.0e-2, max_evl=40).to(device)
max_force, dE =  opt.run(molecule)

Step,  Max_Force,      Etot(eV),     dE(eV)
1      1.219416e+00 ||-4.402169e+02 -4.402169e+02 ||-4.402169e+02 -4.402169e+02 ||-3.189865e+02 -3.189865e+02 
2      1.014750e+00 ||-4.402244e+02 -7.516916e-03 ||-4.402244e+02 -7.516916e-03 ||-3.190117e+02 -2.520887e-02 
3      8.513715e-01 ||-4.402275e+02 -3.082785e-03 ||-4.402275e+02 -3.082785e-03 ||-3.190291e+02 -1.740418e-02 
4      7.188886e-01 ||-4.402291e+02 -1.595697e-03 ||-4.402291e+02 -1.595697e-03 ||-3.190414e+02 -1.231480e-02 
5      6.102487e-01 ||-4.402300e+02 -8.545348e-04 ||-4.402300e+02 -8.545348e-04 ||-3.190503e+02 -8.882842e-03 
6      5.204406e-01 ||-4.402304e+02 -4.601459e-04 ||-4.402304e+02 -4.601459e-04 ||-3.190568e+02 -6.501733e-03 
7      4.456788e-01 ||-4.402307e+02 -2.481912e-04 ||-4.402307e+02 -2.481912e-04 ||-3.190616e+02 -4.818269e-03 
8      3.830912e-01 ||-4.402308e+02 -1.342557e-04 ||-4.402308e+02 -1.342557e-04 ||-3.190652e+02 -3.610380e-03 
9      3.304350e-01 ||-4.402309e+02 -7.309691e-05 ||-4.402309e+02 -7

### Final forces and optimized geometry

In [7]:
molecule.force

tensor([[[ 7.3700e-04, -1.8319e-15, -2.0022e-16],
         [ 7.7385e-04, -1.7587e-14,  8.2331e-16],
         [-7.5543e-04,  1.2139e-03, -1.2276e-15],
         [-7.5543e-04, -1.2139e-03,  6.0452e-16]],

        [[ 7.3700e-04, -1.8319e-15, -2.0022e-16],
         [ 7.7385e-04, -1.7587e-14,  8.2331e-16],
         [-7.5543e-04,  1.2139e-03, -1.2276e-15],
         [-7.5543e-04, -1.2139e-03,  6.0452e-16]],

        [[-5.5576e-03,  5.8197e-03, -6.8407e-03],
         [-3.8264e-03, -5.5583e-03,  6.5334e-03],
         [ 9.3840e-03, -2.6143e-04,  3.0729e-04],
         [-0.0000e+00, -0.0000e+00, -0.0000e+00]]])

In [8]:
molecule.coordinates

Parameter containing:
tensor([[[ 1.5828e-02,  4.8139e-16,  1.7967e-16],
         [ 1.2266e+00, -4.0215e-16, -1.0548e-16],
         [ 1.8088e+00,  9.3016e-01,  5.2400e-16],
         [ 1.8088e+00, -9.3016e-01, -5.9819e-16]],

        [[ 1.5828e-02,  4.8139e-16,  1.7967e-16],
         [ 1.2266e+00, -4.0215e-16, -1.0548e-16],
         [ 1.8088e+00,  9.3016e-01,  5.2400e-16],
         [ 1.8088e+00, -9.3016e-01, -5.9819e-16]],

        [[-2.6290e-02,  1.5550e-02, -1.8278e-02],
         [ 9.1952e-01, -3.1598e-02,  3.7142e-02],
         [-3.8323e-01, -5.5395e-01,  6.5114e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00]]], requires_grad=True)

### Save optimized geometries to .xyz files without final forces.

In [9]:
save_xyz(molecule, 'XYZ', Forces=False)

## Reading starting geometries from .xyz
### Only molecules of the same length are supported for batched xyz_reader.

In [10]:
%%time
torch.manual_seed(0)
files = ['coronene.xyz', 'coronene.xyz']

species, coordinates = read_xyz(files)
species = torch.as_tensor(species,dtype=torch.int64, device=device)[:]
coordinates = torch.tensor(coordinates, device=device)[:]
const = Constants().to(device)
#may need to add scaling factor for length and energy on const, check constants.py

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM3, PM6, PM6_SP. PM6_SP is PM6 without d-orbitals. Effectively, PM6 for the first two rows of periodic table
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

opt =  Geometry_Optimization_SD(seqm_parameters, alpha=0.007, force_tol=1.0e-2, max_evl=60).to(device)
max_force, dE =  opt.run(molecule)

Step,  Max_Force,      Etot(eV),     dE(eV)
1      6.242161e-01 ||-3.237053e+03 -3.237053e+03 ||-3.237053e+03 -3.237053e+03 
2      2.846753e-01 ||-3.237106e+03 -5.207988e-02 ||-3.237106e+03 -5.207988e-02 
3      1.642115e-01 ||-3.237114e+03 -8.720791e-03 ||-3.237114e+03 -8.720791e-03 
4      1.090901e-01 ||-3.237117e+03 -3.229136e-03 ||-3.237117e+03 -3.229136e-03 
5      7.972116e-02 ||-3.237119e+03 -1.526681e-03 ||-3.237119e+03 -1.526681e-03 
6      6.578258e-02 ||-3.237120e+03 -8.114308e-04 ||-3.237120e+03 -8.114309e-04 
7      5.700082e-02 ||-3.237120e+03 -4.873348e-04 ||-3.237120e+03 -4.873348e-04 
8      5.190370e-02 ||-3.237121e+03 -3.309183e-04 ||-3.237121e+03 -3.309183e-04 
9      4.919277e-02 ||-3.237121e+03 -2.542498e-04 ||-3.237121e+03 -2.542498e-04 
10      4.607878e-02 ||-3.237121e+03 -2.128771e-04 ||-3.237121e+03 -2.128771e-04 
11      4.399420e-02 ||-3.237121e+03 -1.838383e-04 ||-3.237121e+03 -1.838383e-04 
12      4.213327e-02 ||-3.237121e+03 -1.655425e-04 ||-3.237121e

### Alternatively, use zero-padding in .xyz files for molecules of different lengths.

In [11]:
%%time
torch.manual_seed(0)
files = ['coronene.xyz', 'benzene_zero_pad.xyz']

species, coordinates = read_xyz(files)
species = torch.as_tensor(species,dtype=torch.int64, device=device)[:]
coordinates = torch.tensor(coordinates, device=device)[:]
const = Constants().to(device)
#may need to add scaling factor for length and energy on const, check constants.py

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM3, PM6, PM6_SP. PM6_SP is PM6 without d-orbitals. Effectively, PM6 for the first two rows of periodic table
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

opt =  Geometry_Optimization_SD(seqm_parameters, alpha=0.007, force_tol=1.0e-2, max_evl=60).to(device)
max_force, dE =  opt.run(molecule)

Step,  Max_Force,      Etot(eV),     dE(eV)
1      6.242236e-01 ||-3.237053e+03 -3.237053e+03 ||-8.503173e+02 -8.503173e+02 
2      2.846696e-01 ||-3.237106e+03 -5.207994e-02 ||-8.503303e+02 -1.305697e-02 
3      1.721282e-01 ||-3.237114e+03 -8.720733e-03 ||-8.503324e+02 -2.044671e-03 
4      1.387342e-01 ||-3.237117e+03 -3.229139e-03 ||-8.503336e+02 -1.214895e-03 
5      1.147022e-01 ||-3.237119e+03 -1.526679e-03 ||-8.503344e+02 -8.351382e-04 
6      9.538084e-02 ||-3.237120e+03 -8.114295e-04 ||-8.503350e+02 -5.785304e-04 
7      7.943185e-02 ||-3.237120e+03 -4.873330e-04 ||-8.503354e+02 -4.012143e-04 
8      6.595378e-02 ||-3.237121e+03 -3.310854e-04 ||-8.503357e+02 -2.779792e-04 
9      5.515255e-02 ||-3.237121e+03 -2.542099e-04 ||-8.503359e+02 -1.929875e-04 
10      4.597942e-02 ||-3.237121e+03 -2.125263e-04 ||-8.503360e+02 -1.341497e-04 
11      4.395084e-02 ||-3.237121e+03 -1.834084e-04 ||-8.503361e+02 -9.272896e-05 
12      4.211763e-02 ||-3.237121e+03 -1.655973e-04 ||-8.503362e